
# Competition2 Report
### Group 43

## 1. Student ID, name of each team member.


107065511 莊筑鈞

107062523 程薰瑩

## 2. Your code

dataset.py

In [ ]:
import os
import math
import random
import cv2
import numpy as np
import tensorflow as tf
import re
import sys
import time
import random
#if you have multiple GPU on the machine, choose only one to use on this notebook
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#let the gpu allocates memory space dynamically
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
import matplotlib.pyplot as plt

classes_name =  ["aeroplane", "bicycle", "bird", "boat", "bottle", 
                 "bus", "car", "cat", "chair", "cow", "diningtable", 
                 "dog", "horse", "motorbike", "person", "pottedplant", 
                 "sheep", "sofa", "train","tvmonitor"]
### DATASET LOADER ###
class DatasetRunner:
    """
    Load pascalVOC 2007 dataset and creates an input pipeline ready to be fed into a model.
    - Reshapes images into 448 x 448
    - converts [0 1] to [-1 1]
    - shuffles the input
    - builds batches
    """
    def __init__(self, common_params, dataset_params, config): 
        self.width = common_params['image_size']
        self.height = common_params['image_size']
        self.batch_size = common_params['batch_size']
        self.num_classes = common_params['num_classes']
        self.data_path = dataset_params['path']
        self.thread_num = dataset_params['thread_num']
        self.image_dir = dataset_params['image_dir']
        self.max_objects = common_params['max_objects_per_image']
        self.graph = tf.Graph()
        self.sess = tf.Session(graph=self.graph, config=config)
        self.image_names = []
        self.record_list = []
        self.object_num_list = []
        # filling the record_list
        input_file = open(self.data_path, 'r')
        for line in input_file:
            line = line.strip()
            ss = line.split(' ')
            self.image_names.append(ss[0])
            self.record_list.append([float(num) for num in ss[1:]])
            self.object_num_list.append(min(len(self.record_list[-1])//5, self.max_objects))
            if len(self.record_list[-1])<self.max_objects*5:
                self.record_list[-1] = self.record_list[-1] +[float(0), float(0), float(0), float(0), float(0)]*\
                (self.max_objects-len(self.record_list[-1])//5)
            elif len(self.record_list[-1])>self.max_objects*5:
                self.record_list[-1] = self.record_list[-1][:self.max_objects*5]
        self.build_train_data_tensor()
          
    def build_train_data_tensor(self):       
        def data_generator(image_name, raw_labels, object_num):
            image_file = tf.read_file(self.image_dir+image_name)
            image = tf.image.decode_jpeg(image_file, channels=3)
            h = tf.shape(image)[0]
            w = tf.shape(image)[1]
            width_rate = self.width * 1.0 / tf.cast(w, tf.float32) 
            height_rate = self.height * 1.0 / tf.cast(h, tf.float32)
            image = tf.image.resize_images(image, size=[self.height,self.width])
            image = image / 255 * 2 - 1
            #image = tf.image.random_hue(image, max_delta=0.2)
            #image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
            #image = tf.image.random_brightness(image, max_delta=32./255.)
            #image = tf.image.random_saturation(image, lower=0.5, upper=1.5) 
            raw_labels = tf.cast(tf.reshape(raw_labels, [-1, 5]), tf.float32)
            xmin = raw_labels[:, 0]
            ymin = raw_labels[:, 1]
            xmax = raw_labels[:, 2]
            ymax = raw_labels[:, 3]
            class_num = raw_labels[:, 4]
            xcenter = (xmin + xmax) * 1.0 / 2.0 * width_rate
            ycenter = (ymin + ymax) * 1.0 / 2.0 * height_rate
            box_w = (xmax - xmin) * width_rate
            box_h = (ymax - ymin) * height_rate
            #th = random.random()
            #if th > 0.5:
            #    image = tf.image.flip_left_right(image)
            #    xcenter = self.width - 1 - xcenter 
            labels = tf.stack([xcenter, ycenter, box_w, box_h, class_num], axis = 1)
            return image, labels, tf.cast(object_num, tf.int32)
            
        with self.graph.as_default():
            dataset = tf.data.Dataset.from_tensor_slices((self.image_names, 
                                                          np.array(self.record_list), 
                                                          np.array(self.object_num_list)))
            #dataset = dataset.shuffle(len(self.record_list))
            dataset = dataset.map(data_generator, num_parallel_calls = self.thread_num)
            #dataset = dataset.batch(self.batch_size)
            dataset = dataset.shuffle(len(self.record_list)) 
            dataset = dataset.batch(self.batch_size)
            dataset = dataset.repeat()
            self.iterator = tf.data.Iterator.from_structure(dataset.output_types)  
            self.data_init_op = self.iterator.make_initializer(dataset)
            self.sess.run(self.data_init_op)
            self.iterate_op = self.iterator.get_next()
            
    def batch(self):
        images, labels, objects_num = self.sess.run(self.iterate_op)
        if objects_num.shape[0] < self.batch_size:
            images, labels, objects_num = self.sess.run(self.iterate_op)
        #images = images/255 * 2 - 1
        return images, labels, objects_num
        
    def __del__(self):
        self.close()

    def close(self):
        self.sess.close()

model_def.py

In [ ]:
import os, sys, time
import tensorflow as tf
import numpy as np

class YoloRunner(object):
    def __init__(self, dataset, net, common_params, solver_params, pre_model=False):
        #process params
        self.learning_rate = float(solver_params['learning_rate'])
        #self.decay_rate = float(solver_params['decay_rate'])
        self.moment = float(solver_params['moment'])
        self.batch_size = int(common_params['batch_size'])
        self.height = int(common_params['image_size'])
        self.width = int(common_params['image_size'])
        self.max_objects = int(common_params['max_objects_per_image'])
        self.train_dir = str(solver_params['train_dir'])
        if pre_model:
            self.pretrain_path = str(solver_params['pretrain_path'])
        if not os.path.exists(self.train_dir):
            os.makedirs(self.train_dir)
        self.max_iterators = int(solver_params['max_iterators'])
        #self.decay_step = int(solver_params['decay_step'])
        self.print_frequency = int(solver_params['print_frequency'])
        self.save_frequency = int(solver_params['save_frequency'])
        self.dataset = dataset
        self.net = net
        self.construct_graph()

    def _train(self):
        """Train model
        Create an optimizer and apply to all trainable variables.
        Args:
          total_loss: Total loss from net.loss()
          global_step: Integer Variable counting the number of training steps
          processed
        Returns:
          train_op: op for training
        """
        #opt = tf.train.MomentumOptimizer(self.learning_rate, self.moment)
        #opt = tf.train.GradientDescentOptimizer(self.learning_rate)
        self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        #print(update_ops)
        #with tf.control_dependencies(self.update_ops):
        #lr = tf.train.exponential_decay(self.learning_rate, self.global_step, self.decay_step, self.decay_rate, True)
        opt = tf.train.AdamOptimizer(self.learning_rate)
        grads = opt.compute_gradients(self.total_loss) #tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))
        apply_gradient_op = opt.apply_gradients(grads, global_step=self.global_step)
        return apply_gradient_op

    def construct_graph(self):
        self.global_step = tf.Variable(0, trainable=False)        
        self.images = tf.placeholder(tf.float32, (None, self.height, self.width, 3))
        self.labels = tf.placeholder(tf.float32, (None, self.max_objects, 5))
        self.objects_num = tf.placeholder(tf.int32, (None))
        self.predicts = self.net.inference(self.images)
        self.total_loss, self.nilboy, self.losses = self.net.loss(self.predicts, self.labels, self.objects_num)
        tf.summary.scalar('loss', self.total_loss)
        self.train_op = self._train()

    def run(self, config, pre_model=False):
        #self.construct_graph()
       # v_list = tf.trainable_varables() 
       # g_list = tf.global_variables()
        #bn_moving_vars = [g for g in g_list if ('moving_mean' or 'moving_variance' or 'gamma' or 'beta') in g.name]
        #bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        #bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        #bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        #v_list += bn_moving_vars
        #print(bn_moving_vars)
        #self.net.trainable_collection.append(bn_moving_vars)
        #saver = tf.train.Saver(v_list, write_version=tf.train.SaverDef.V2)
        init =  tf.global_variables_initializer()
        summary_op = tf.summary.merge_all()
        sess = tf.Session(config = config)
        sess.run(init)
        #self.train_op = self._train()
        v_list = tf.trainable_variables() 
        g_list = tf.global_variables()
        #bn_moving_vars = [g for g in g_list if ('moving_mean' or 'moving_variance' or 'gamma' or 'beta') in g.name]
        #bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        v_list += bn_moving_vars
        #print(bn_moving_vars)
        #self.net.trainable_collection.append(bn_moving_vars)
        saver = tf.train.Saver(self.net.trainable_collection, write_version=tf.train.SaverDef.V2)
        #self.train_op = self._train()
        if pre_model:
            saver.restore(sess, self.pretrain_path)
            print('Finish Loading Pre-trained model: %s' %self.pretrain_path)
        summary_writer = tf.summary.FileWriter(self.train_dir, sess.graph)
        for step in range(self.max_iterators):
            start_time = time.time()
            np_images, np_labels, np_objects_num = self.dataset.batch()
            _, loss_value, losses, nilboy= sess.run([self.train_op, self.total_loss, self.losses, self.nilboy], 
                                            feed_dict={self.images: np_images, 
                                                       self.labels: np_labels,
                                                       self.objects_num: np_objects_num})
            duration = time.time() - start_time
            assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
            if step % self.print_frequency== 0:
                num_examples_per_step = self.dataset.batch_size
                examples_per_sec = num_examples_per_step / duration
                sec_per_batch = float(duration)
                format_str = ('step %d, loss = %.2f, cls_loss = %.2f, obj_loss = %.2f, nobj_loss = %.2f, coord_loss = %.2f (%.1f examples/sec; %.3f ' 'sec/batch)')
                print (format_str % (step, loss_value, losses[0], losses[1], losses[2], losses[3], examples_per_sec, sec_per_batch))
                sys.stdout.flush()
            if step % self.print_frequency == 0:
                summary_str = sess.run(summary_op, feed_dict={self.images: np_images, 
                                                              self.labels: np_labels, 
                                                              self.objects_num: np_objects_num})
                summary_writer.add_summary(summary_str, step)
            if step % self.save_frequency == 0:
                saver.save(sess, self.train_dir + '/model.ckpt', global_step=step)
        sess.close()

    def prepare_inference(self, model_version, config):
        saver = tf.train.Saver(self.net.trainable_collection)
        init =  tf.global_variables_initializer()
        self.sess = tf.Session(config = config)
        self.sess.run(init)
        v_list = tf.trainable_variables() 
        g_list = tf.global_variables()
        #bn_moving_vars = [g for g in g_list if ('moving_mean' or 'moving_variance' or 'gamma' or 'beta') in g.name]
        #bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        #bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        #bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        #v_list += bn_moving_vars
        #saver = tf.train.Saver(v_list)
        saver.restore(self.sess, self.train_dir+'/model.ckpt-'+model_version)

    def make_one_prediction(self, images):
        prediction = self.sess.run(self.predicts, feed_dict= {self.images: images})
        return prediction

yolov2.py

In [ ]:
import os
import math
import random
import cv2
import numpy as np
import tensorflow as tf
import re
import sys
import time
#if you have multiple GPU on the machine, choose only one to use on this notebook
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#let the gpu allocates memory space dynamically
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
import matplotlib.pyplot as plt
#from dataset import *

class YoloTinyNet(object):

    def __init__(self, common_params, net_params, is_train=True):
        """
        common params: a params dict
        net_params   : a params dict
        """
        #pretrained variable collection
        self.pretrained_collection = []
        #trainable variable collection
        self.trainable_collection = [] 
        #process params
        self.image_size = int(common_params['image_size'])
        self.num_classes = int(common_params['num_classes'])
        self.cell_size = int(net_params['cell_size'])
        self.boxes_per_cell = int(net_params['boxes_per_cell'])
        self.batch_size = int(common_params['batch_size'])
        self.weight_decay = float(net_params['weight_decay'])
        #if not test:
        self.is_train = is_train
        self.object_scale = float(net_params['object_scale'])
        self.noobject_scale = float(net_params['noobject_scale'])
        self.class_scale = float(net_params['class_scale'])
        self.coord_scale = float(net_params['coord_scale'])
         
    def _variable_on_cpu(self, name, shape, initializer, pretrain=False, train=True):
        """Helper to create a Variable stored on CPU memory.
        Args:
          name: name of the Variable
          shape: list of ints
          initializer: initializer of Variable
        Returns:
          Variable Tensor
        """
        with tf.device('/device:GPU:0'):
            var = tf.get_variable(name, shape, initializer=initializer, dtype=tf.float32)
            if pretrain:
                self.pretrained_collection.append(var)
            if train:
                self.trainable_collection.append(var)
        return var 
    
    def _variable_with_weight_decay(self, name, shape, stddev, wd, pretrain=False, train=True):
        """Helper to create an initialized Variable with weight decay.
        Note that the Variable is initialized with truncated normal distribution
        A weight decay is added only if one is specified.
        Args:
          name: name of the variable 
          shape: list of ints
          stddev: standard devision of a truncated Gaussian
          wd: add L2Loss weight decay multiplied by this float. If None, weight 
          decay is not added for this Variable.
       Returns:
          Variable Tensor 
        """
        var = self._variable_on_cpu(name, shape,
            tf.truncated_normal_initializer(stddev=stddev, dtype=tf.float32), pretrain, train)
        if wd is not None:
            weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
            tf.add_to_collection('losses', weight_decay)
        return var
 
    def conv_block(self, inp, scope, kernel_width, kernel_height, inp_channel, out_channel, strides = [1, 1, 1, 1], padding='SAME', leaky=True, pretrain=False, train=True):
        with tf.variable_scope(scope) as scope:
            kernel = self._variable_with_weight_decay('weights', [kernel_width, kernel_width, inp_channel, out_channel], 0.01, self.weight_decay, pretrain, train)
            #biases = self._variable_on_cpu('bias', [out_channel], tf.constant_initializer(0.0), pretrain, train)
            local = tf.nn.conv2d(inp, kernel, strides=strides, padding=padding)
            #local = tf.layers.batch_normalization(conv, training=self.is_train, momentum=0.9, name=scope.name)
            #print(local)
            #if pretrain:
            #    self.pretrained_collection.append(local.variables)
                #self.pretrained_collection.append(local.moving_mean)
            #if train:
            #    self.trainable_collection.append(local.variables)
                #self.trainable_collection.append(local.moving_variance)
            #local = tf.nn.bias_add(conv, biases)
            #return self.leaky_relu(pre_activation, name=scope.name)
            if leaky:
                local = self.leaky_relu(local, name=scope.name)
            else:
                local = tf.identity(local, name=scope.name)
        return local

    def fully(self, scope, input, in_dimension, out_dimension, leaky=True, pretrain=False, train=True):
        """Fully connection layer
        Args:
          scope: variable_scope name
          input: [batch_size, ???]
          out_dimension: int32
        Return:
          output: 2-D tensor [batch_size, out_dimension]
        """
        with tf.variable_scope(scope) as scope:
            reshape = tf.reshape(input, [tf.shape(input)[0], -1])
            weights = self._variable_with_weight_decay('weights', shape=[in_dimension, out_dimension],
                                  stddev=0.01, wd=self.weight_decay, pretrain=pretrain, train=train)
            biases = self._variable_on_cpu('biases', [out_dimension], tf.constant_initializer(0.0), pretrain, train)
            local = tf.matmul(reshape, weights) + biases
            if leaky:
                local = self.leaky_relu(local, name=scope.name)
            else:
                local = tf.identity(local, name=scope.name)
        return local

    def leaky_relu(self, x, name, alpha=0.1, dtype=tf.float32):
        """leaky relu 
        if x > 0:
          return x
        else:
          return alpha * x
        Args:
          x : Tensor
          alpha: float, the slope of the leaky function
        Return:
          y : Tensor
        """
        x = tf.cast(x, dtype=dtype)
        return tf.nn.leaky_relu(x, alpha=alpha, name=name)
    
    def inference(self, images):
        """Build the yolo model
        Input the images, output prediction boxes(center_x, center_y, w, h, scale) and the corresponding classes
        
        Args:
          images:  4-D tensor [batch_size, image_height, image_width, channels]
        Returns:
          predicts: 4-D tensor [batch_size, cell_size, cell_size, num_classes + 5 * boxes_per_cell]
        """
        #images = tf.reshape(images, [-1, self.image_size*self.image_size*3])
        #convolutional layers
        h_conv1 = self.conv_block(images, "conv1", 7, 7, 3, 64, [1, 2, 2, 1])
        pool_1 = tf.nn.max_pool(h_conv1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
        h_conv2 = self.conv_block(pool_1, "conv2", 3, 3, 64, 192)
        pool_2 = tf.nn.max_pool(h_conv2, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
        res3_1 = self.conv_block(pool_2, 'res3_1', 1, 1, 192, 256)
        h_conv3_1 = self.conv_block(pool_2, "conv3_1", 1, 1, 192, 128)
        h_conv3_2 = self.conv_block(h_conv3_1, "conv3_2", 3, 3, 128, 256)
        h_conv3_2 = tf.add(h_conv3_2, res3_1)
        #h_conv3_2 = self.leaky_relu(h_conv3_2, 'conv32')
        res3_2 = self.conv_block(h_conv3_2, 'res3_2', 1, 1, 256, 512)
        h_conv3_3 = self.conv_block(h_conv3_2, "conv3_3", 1, 1, 256, 256)
        h_conv3_4 = self.conv_block(h_conv3_3, "conv3_4", 3, 3, 256, 512)
        h_conv3_4 = tf.add(h_conv3_4, res3_2)
        #h_conv3_4 = self.leaky_relu(h_conv3_4, 'conv34')
        pool_3 = tf.nn.max_pool(h_conv3_4, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
        h_conv4_1 = self.conv_block(pool_3, "conv4_1", 1, 1, 512, 256)
        h_conv4_2 = self.conv_block(h_conv4_1, "conv4_2", 3, 3, 256, 512)
        h_conv4_2 = tf.add(h_conv4_2, pool_3)
        #h_conv4_2 = self.leaky_relu(h_conv4_2, 'conv42')
        h_conv4_3 = self.conv_block(h_conv4_2, "conv4_3", 1, 1, 512, 256)
        h_conv4_4 = self.conv_block(h_conv4_3, "conv4_4", 3, 3, 256, 512)
        h_conv4_4 = tf.add(h_conv4_4, h_conv4_2)
        #h_conv4_4 = self.leaky_relu(h_conv4_4, 'conv44')
        h_conv4_5 = self.conv_block(h_conv4_4, "conv4_5", 1, 1, 512, 256)
        h_conv4_6 = self.conv_block(h_conv4_5, "conv4_6", 3, 3, 256, 512)
        h_conv4_6 = tf.add(h_conv4_6, h_conv4_4)
        #h_conv4_6 = self.leaky_relu(h_conv4_6, 'conv46')
        h_conv4_7 = self.conv_block(h_conv4_6, "conv4_7", 1, 1, 512, 256)
        h_conv4_8 = self.conv_block(h_conv4_7, "conv4_8", 3, 3, 256, 512)
        h_conv4_8 = tf.add(h_conv4_8, h_conv4_6)
        #h_conv4_8 = self.leaky_relu(h_conv4_8, 'conv48')
        res4_1 = self.conv_block(h_conv4_8, 'res4_1', 1, 1, 512, 1024)
        h_conv4_9 = self.conv_block(h_conv4_8, "conv4_9", 1, 1, 512, 512)
        h_conv4_10 = self.conv_block(h_conv4_9, "conv4_10", 3, 3, 512, 1024)
        h_conv4_10 = tf.add(h_conv4_10, res4_1)
        #h_conv4_10 = self.leaky_relu(h_conv4_10, 'conv410')
        pool_4 = tf.nn.max_pool(h_conv4_10, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
        h_conv5_1 = self.conv_block(pool_4, "conv5_1", 1, 1, 1024, 512)
        h_conv5_2 = self.conv_block(h_conv5_1, "conv5_2", 3, 3, 512, 1024)
        h_conv5_2 = tf.add(h_conv5_2, pool_4)
        #h_conv5_2 = self.leaky_relu(h_conv5_2, 'conv52')
        h_conv5_3 = self.conv_block(h_conv5_2, "conv5_3", 1, 1, 1024, 512)
        h_conv5_4 = self.conv_block(h_conv5_3, "conv5_4", 3, 3, 512, 1024)
        h_conv5_4 = tf.add(h_conv5_2, h_conv5_4)
        #h_conv5_4 = self.leaky_relu(h_conv5_4, 'conv54')
        h_conv5_5 = self.conv_block(h_conv5_4, "conv5_5", 3, 3, 1024, 1024)
        h_conv5_6 = self.conv_block(h_conv5_5, "conv5_6", 3, 3, 1024, 1024, strides=[1, 2, 2, 1])
        res5 = tf.nn.max_pool(h_conv5_4, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')
        h_conv5_6 = tf.add(h_conv5_6, res5)
        #h_conv5_6 = self.leaky_relu(h_conv5_6, 'conv56')
        h_conv6_1 = self.conv_block(h_conv5_6, "conv6_1", 3, 3, 1024, 1024)
        h_conv6_2 = self.conv_block(h_conv6_1, "conv6_2", 3, 3, 1024, 1024)
        h_conv6_2 = tf.add(h_conv6_2, h_conv5_6)
        #h_conv6_2 = self.leaky_relu(h_conv6_2, 'conv62')
        #fully0 = self.fully('local0', images, self.image_size*self.image_size*3, 256)
        #fully1 = self.fully('local1', fully0, 256, 512)
        #flatten = tf.contrib.layers.flatten(h_conv6_2, scope='flatten')
        #h_conv6_2 = tf.transpose(h_conv6_2, [0, 3, 1, 2])
        #fully0 = self.fully('fully0', h_conv6_2, 7*7*1024, 512)
        fully1 = self.fully('fully1', h_conv6_2, 7*7*1024, 4096)
        #fully1 = tf.nn.dropout(fully1, 0.5)
        fully2 = self.fully('fully2', fully1, 4096, self.cell_size * self.cell_size * 
                            (self.num_classes + self.boxes_per_cell * 5), leaky=False,
                            pretrain=False, train=True)
        n1 = self.cell_size * self.cell_size * self.num_classes
        n2 = n1 + self.cell_size * self.cell_size * self.boxes_per_cell
        class_probs = tf.reshape(fully2[:, 0:n1], (-1, self.cell_size, self.cell_size, self.num_classes))
        scales = tf.reshape(fully2[:, n1:n2], (-1, self.cell_size, self.cell_size, self.boxes_per_cell))
        boxes = tf.reshape(fully2[:, n2:], (-1, self.cell_size, self.cell_size, self.boxes_per_cell * 4))
        predicts = tf.concat([class_probs, scales, boxes], 3)
        return predicts

    def iou(self, boxes1, boxes2):
        """calculate ious
        Args:
          boxes1: 4-D tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 4]  ====> (x_center, y_center, w, h)
          boxes2: 1-D tensor [4] ===> (x_center, y_center, w, h)
          
        Return:
          iou: 3-D tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
        """
        #boxes1 : [4(xmin, ymin, xmax, ymax), cell_size, cell_size, boxes_per_cell]
        boxes1 = tf.stack([boxes1[:, :, :, 0] - boxes1[:, :, :, 2] / 2, boxes1[:, :, :, 1] - boxes1[:, :, :, 3] / 2,
                          boxes1[:, :, :, 0] + boxes1[:, :, :, 2] / 2, boxes1[:, :, :, 1] + boxes1[:, :, :, 3] / 2])
        #boxes1 : [cell_size, cell_size, boxes_per_cell, 4(xmin, ymin, xmax, ymax)]
        boxes1 = tf.transpose(boxes1, [1, 2, 3, 0])
        boxes2 =  tf.stack([boxes2[0] - boxes2[2] / 2, boxes2[1] - boxes2[3] / 2,
                            boxes2[0] + boxes2[2] / 2, boxes2[1] + boxes2[3] / 2])
        #calculate the left up point of boxes' overlap area
        lu = tf.maximum(boxes1[:, :, :, 0:2], boxes2[0:2])
        #calculate the right down point of boxes overlap area
        rd = tf.minimum(boxes1[:, :, :, 2:], boxes2[2:])
        #intersection
        intersection = rd - lu 
        #the size of the intersection area
        inter_square = intersection[:, :, :, 0] * intersection[:, :, :, 1]
        mask = tf.cast(intersection[:, :, :, 0] > 0, tf.float32) * tf.cast(intersection[:, :, :, 1] > 0, tf.float32)
        #if intersection is negative, then the boxes don't overlap
        inter_square = mask * inter_square
        #calculate the boxs1 square and boxs2 square
        square1 = (boxes1[:, :, :, 2] - boxes1[:, :, :, 0]) * (boxes1[:, :, :, 3] - boxes1[:, :, :, 1])
        square2 = (boxes2[2] - boxes2[0]) * (boxes2[3] - boxes2[1])
        return inter_square/(square1 + square2 - inter_square + 1e-6)

    def losses_calculation(self, num, object_num, loss, predict, labels, nilboy):
        """
        calculate loss
        Args:
          predict: 3-D tensor [cell_size, cell_size, 5 * boxes_per_cell]
          labels : [max_objects, 5]  (x_center, y_center, w, h, class)
        """
        label = labels[num:num+1, :]
        label = tf.reshape(label, [-1])
        #calculate objects  tensor [CELL_SIZE, CELL_SIZE]
        min_x = (label[0] - label[2] / 2) / (self.image_size / self.cell_size)
        max_x = (label[0] + label[2] / 2) / (self.image_size / self.cell_size)
        min_y = (label[1] - label[3] / 2) / (self.image_size / self.cell_size)
        max_y = (label[1] + label[3] / 2) / (self.image_size / self.cell_size)
        min_x = tf.floor(min_x)
        min_y = tf.floor(min_y)
        max_x = tf.minimum(tf.ceil(max_x), self.cell_size)
        max_y = tf.minimum(tf.ceil(max_y), self.cell_size)
        temp = tf.cast(tf.stack([max_y - min_y, max_x - min_x]), dtype=tf.int32)
        objects = tf.ones(temp, tf.float32)
        temp = tf.cast(tf.stack([min_y, self.cell_size - max_y, min_x, self.cell_size - max_x]), tf.int32)
        temp = tf.reshape(temp, (2, 2))
        objects = tf.pad(objects, temp, "CONSTANT")
        #calculate objects  tensor [CELL_SIZE, CELL_SIZE]
        #calculate responsible tensor [CELL_SIZE, CELL_SIZE]
        center_x = label[0] / (self.image_size / self.cell_size)
        center_x = tf.floor(center_x)
        center_y = label[1] / (self.image_size / self.cell_size)
        center_y = tf.floor(center_y)
        response = tf.ones([1, 1], tf.float32)
        temp = tf.cast(tf.stack([center_y, self.cell_size - center_y - 1, 
                                 center_x, self.cell_size -center_x - 1]), tf.int32)
        self.tmp = tf.stack([center_y, self.cell_size - center_y - 1, 
                             center_x, self.cell_size -center_x - 1])
        temp = tf.reshape(temp, (2, 2))
        response = tf.pad(response, temp, "CONSTANT")
        #objects = response
        #calculate iou_predict_truth [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
        predict_boxes = predict[:, :, self.num_classes + self.boxes_per_cell:]
        predict_boxes = tf.reshape(predict_boxes, [self.cell_size, 
                                                   self.cell_size, 
                                                   self.boxes_per_cell, 4])
        predict_boxes = predict_boxes * [self.image_size / self.cell_size, 
                                         self.image_size / self.cell_size, 
                                         self.image_size, self.image_size]
        base_boxes = np.zeros([self.cell_size, self.cell_size, 4])
        #for each cell
        for y in range(self.cell_size):
            for x in range(self.cell_size):
                base_boxes[y, x, :] = [self.image_size / self.cell_size * x, self.image_size / self.cell_size * y, 0, 0]
        base_boxes = np.tile(np.resize(base_boxes, [self.cell_size, self.cell_size, 1, 4]), [1, 1, self.boxes_per_cell, 1])
        #if there's no predict_box in that cell, then the base_boxes will be calcuated with label and got iou equals 0
        predict_boxes = base_boxes + predict_boxes
        iou_predict_truth = self.iou(predict_boxes, label[0:4])
        #calculate C [cell_size, cell_size, boxes_per_cell]
        C = iou_predict_truth * tf.reshape(response, [self.cell_size, self.cell_size, 1])
        #calculate I tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
        I = iou_predict_truth * tf.reshape(response, (self.cell_size, self.cell_size, 1))
        max_I = tf.reduce_max(I, 2, keep_dims=True)
        I = tf.cast((I >= max_I), tf.float32) * tf.reshape(response, (self.cell_size, self.cell_size, 1))
        #calculate no_I tensor [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
        no_I = tf.ones_like(I, dtype=tf.float32) - I 
        p_C = predict[:, :, self.num_classes:self.num_classes + self.boxes_per_cell]
        #calculate truth x, y, sqrt_w, sqrt_h 0-D
        x = label[0]
        y = label[1]
        sqrt_w = tf.sqrt(tf.abs(label[2]))
        sqrt_h = tf.sqrt(tf.abs(label[3]))
        #calculate predict p_x, p_y, p_sqrt_w, p_sqrt_h 3-D [CELL_SIZE, CELL_SIZE, BOXES_PER_CELL]
        p_x = predict_boxes[:, :, :, 0]
        p_y = predict_boxes[:, :, :, 1]
        #p_sqrt_w = tf.sqrt(tf.abs(predict_boxes[:, :, :, 2])) * ((tf.cast(predict_boxes[:, :, :, 2] > 0, tf.float32) * 2) - 1)
        #p_sqrt_h = tf.sqrt(tf.abs(predict_boxes[:, :, :, 3])) * ((tf.cast(predict_boxes[:, :, :, 3] > 0, tf.float32) * 2) - 1)
        #p_sqrt_w = tf.sqrt(tf.maximum(0.0, predict_boxes[:, :, :, 2]))
        #p_sqrt_h = tf.sqrt(tf.maximum(0.0, predict_boxes[:, :, :, 3]))
        #p_sqrt_w = predict_boxes[:, :, :, 2]
        #p_sqrt_h = predict_boxes[:, :, :, 3]
        p_sqrt_w = tf.sqrt(tf.minimum(self.image_size * 1.0, tf.maximum(0.0, predict_boxes[:, :, :, 2])))
        p_sqrt_h = tf.sqrt(tf.minimum(self.image_size * 1.0, tf.maximum(0.0, predict_boxes[:, :, :, 3])))
        #calculate truth p 1-D tensor [NUM_CLASSES]
        P = tf.one_hot(tf.cast(label[4], tf.int32), self.num_classes, dtype=tf.float32)
        #calculate predict p_P 3-D tensor [CELL_SIZE, CELL_SIZE, NUM_CLASSES]
        p_P = predict[:, :, 0:self.num_classes]
        #class_loss
        class_loss = tf.nn.l2_loss(tf.reshape(objects, (self.cell_size, self.cell_size, 1)) * (p_P - P)) * self.class_scale
        #print(class_loss)
        #P_temp = tf.reshape(objects, (self.cell_size, self.cell_size, 1)) * tf.reshape(P, (1, 1, self.num_classes))
        #class_loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=P_temp, logits = tf.reshape(objects, (self.cell_size, self.cell_size, 1)) * p_P))
        #class_loss *= self.class_scale
        #print(class_loss)
        
        #class_loss = tf.nn.l2_loss(tf.reshape(response, (self.cell_size, self.cell_size, 1)) * (p_P - P)) * self.class_scale
        #object_loss
        object_loss = tf.nn.l2_loss(I * (p_C - C)) * self.object_scale
        #object_loss = tf.nn.l2_loss(I * (p_C - (C + 1.0)/2.0)) * self.object_scale
        #noobject_loss
        #noobject_loss = tf.nn.l2_loss(no_I * (p_C - C)) * self.noobject_scale
        noobject_loss = tf.nn.l2_loss(no_I * (p_C)) * self.noobject_scale
        #coord_loss
        coord_loss = (tf.nn.l2_loss(I * (p_x - x)/(self.image_size/self.cell_size)) +
                     tf.nn.l2_loss(I * (p_y - y)/(self.image_size/self.cell_size)) +
                     tf.nn.l2_loss(I * (p_sqrt_w - sqrt_w))/ self.image_size +
                     tf.nn.l2_loss(I * (p_sqrt_h - sqrt_h))/self.image_size) * self.coord_scale
        nilboy = I
        return (num + 1, object_num, [loss[0] + class_loss, 
                                      loss[1] + object_loss, 
                                      loss[2] + noobject_loss,
                                      loss[3] + coord_loss], predict, labels, nilboy)

    def loss(self, predicts, labels, objects_num):
        """Add Loss to all the trainable variables
          Args:
          predicts: 4-D tensor [batch_size, cell_size, cell_size, 5 * boxes_per_cell]
          ===> (num_classes, boxes_per_cell, 4 * boxes_per_cell)
          labels  : 3-D tensor of [batch_size, max_objects, 5]
          objects_num: 1-D tensor [batch_size]
        """
        def condition(num, object_num, loss, predict, label, nilboy):
            """
            if num < object_num
            """
            return num < object_num
        class_loss = tf.constant(0, tf.float32)
        object_loss = tf.constant(0, tf.float32)
        noobject_loss = tf.constant(0, tf.float32)
        coord_loss = tf.constant(0, tf.float32)
        loss = [0, 0, 0, 0]
        for i in range(self.batch_size):
            predict = predicts[i, :, :, :]
            label = labels[i, :, :]
            object_num = objects_num[i]
            nilboy = tf.ones([7,7,2])
            tuple_results = tf.while_loop(condition, self.losses_calculation, 
                                          [tf.constant(0), object_num, 
                                           [class_loss, object_loss, noobject_loss, coord_loss], 
                                           predict, label, nilboy])
            for j in range(4):
                loss[j] = loss[j] + tuple_results[2][j]
            nilboy = tuple_results[5]
        tf.add_to_collection('losses', (loss[0] + loss[1] + loss[2] + loss[3])/self.batch_size)
        tf.summary.scalar('class_loss', loss[0]/self.batch_size)
        tf.summary.scalar('object_loss', loss[1]/self.batch_size)
        tf.summary.scalar('noobject_loss', loss[2]/self.batch_size)
        tf.summary.scalar('coord_loss', loss[3]/self.batch_size)
        tf.summary.scalar('weight_loss', tf.add_n(tf.get_collection('losses')) 
                          - (loss[0] + loss[1] + loss[2] + loss[3])/self.batch_size )
        return tf.add_n(tf.get_collection('losses'), name='total_loss'), nilboy, tf.convert_to_tensor(loss)/self.batch_size



train.py

In [ ]:
import os ,sys
print(0)
import tensorflow as tf
print(1)
from dataset import DatasetRunner
print(2)
from model_def import YoloRunner
print(3)
from yolov2 import YoloTinyNet

##### SET HYPERPARAMETERS #####
common_params = {
    'image_size': 448,
    'batch_size': 32,
    'num_classes': 20,
    'max_objects_per_image': 20
}
dataset_params = {
    'path': 'data/pascal_voc_training_data.txt',
    'image_dir': './data/VOCdevkit_train/VOC2007/JPEGImages/',
    'thread_num': 6
}
net_params = {
    'weight_decay': 0.0005,
    'cell_size': 7,
    'boxes_per_cell': 2,
    'object_scale': 1,
    'noobject_scale': 0.5,
    'class_scale': 2,
    'coord_scale': 5, 

}
solver_params = {
    'learning_rate': 0.0001,
    'decay_step': 20000,
    'decay_rate': 0.1,
    'moment': 0.9,
    'max_iterators': 20001,
    'print_frequency': 100,
    'save_frequency' : 1000,
    'train_dir': 'models/yolo-like',
    'pretrain_path': 'models/yolo-like/model.ckpt-25000'
}
if __name__ == '__main__':
    #if you have multiple GPU on the machine, choose only one to use on this notebook
    print(0)
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"
    #let the gpu allocates memory space dynamically
    print('Config')
    #config = tf.ConfigProto()
    #config.gpu_options.allow_growth = True
    #from yolov2 import *
    ##### TRAINING #####
    print('Start!')
    tf.reset_default_graph()
    sys.path.append('./')
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    dataset = DatasetRunner(common_params, dataset_params, config)
    print('Dataset Ready')
    net = YoloTinyNet(common_params, net_params)
    print('Define Network')
    model_runner = YoloRunner(dataset, net, common_params, solver_params)
    print('Model Runner')
    print('Learning Rate: 0.0001')
    model_runner.run(config)

test.py

In [ ]:
import os, sys, time
import numpy as np
import tensorflow as tf
from model_def import YoloRunner
from yolov2 import YoloTinyNet

##### SET HYPERPARAMETERS #####
common_params = {
    'image_size': 448,
    'batch_size': 32,
    'num_classes': 20,
    'max_objects_per_image': 20
}
dataset_params = {
    'path': 'data/pascal_voc_training_data.txt',
    'image_dir': './data/VOCdevkit_train/VOC2007/JPEGImages/',
    'thread_num': 6
}
net_params = {
    'weight_decay': 0.0005,
    'cell_size': 7,
    'boxes_per_cell': 2,
    'object_scale': 1,
    'noobject_scale': 0.5,
    'class_scale': 1,
    'coord_scale': 5, 

}
solver_params = {
    'learning_rate': 0.00001,
    'moment': 0.9,
    'max_iterators': 20001,
    'print_frequency': 100,
    'save_frequency' : 1000,
    'train_dir': 'models',
    'pretrain_path': 'models/yolov2-like/model.ckpt-13000'
}

def process_predicts(predicts):
    """
    Process YOLO outputs into bou
    """
    p_classes = predicts[0, :, :, 0:20]
    C = predicts[0, :, :, 20:22]
    coordinate = predicts[0, :, :, 22:]
    p_classes = np.reshape(p_classes, (net_params['cell_size'], net_params['cell_size'], 1, 20))
    C = np.reshape(C, (net_params['cell_size'], net_params['cell_size'], net_params['boxes_per_cell'], 1))
    P = C * p_classes
    #P's shape [7, 7, 2, 20]
    #print P[5,1, 0, :]
    #choose the most confidence one
    max_conf = np.max(P)
    index = np.argmax(P)
    index = np.unravel_index(index, P.shape)
    class_num = index[3]
    coordinate = np.reshape(coordinate, 
                            (net_params['cell_size'], 
                             net_params['cell_size'], 
                             net_params['boxes_per_cell'], 
                             4))
    max_coordinate = coordinate[index[0], index[1], index[2], :]
    xcenter = max_coordinate[0]
    ycenter = max_coordinate[1]
    w = max_coordinate[2]
    h = max_coordinate[3]
    xcenter = (index[1] + xcenter) * (common_params['image_size']/float(net_params['cell_size']))
    ycenter = (index[0] + ycenter) * (common_params['image_size']/float(net_params['cell_size']))
    w = w * common_params['image_size']
    h = h * common_params['image_size']
    xmin = xcenter - w/2.0
    ymin = ycenter - h/2.0
    xmax = xmin + w
    ymax = ymin + h
    return xmin, ymin, xmax, ymax, class_num, max_conf

#if you have multiple GPU on the machine, choose only one to use on this notebook
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#let the gpu allocates memory space dynamically
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

tf.reset_default_graph()
common_params['batch_size'] = 1
net = YoloTinyNet(common_params, net_params)
solver = YoloRunner(None, net, common_params, solver_params)

#### BUILD TEST DATASET ITERLATOR #####
test_img_files = open('data/pascal_voc_testing_data.txt')
test_img_dir = './data/VOCdevkit_test/VOC2007/JPEGImages/'
test_images = []
img_size = common_params['image_size']
for line in test_img_files:
    line = line.strip()
    ss = line.split(' ')
    test_images.append(ss[0])
test_dataset = tf.data.Dataset.from_tensor_slices(test_images)
def load_img_data(image_name):
    image_file = tf.read_file(test_img_dir+image_name)
    image = tf.image.decode_jpeg(image_file, channels=3)
    h = tf.shape(image)[0]
    w = tf.shape(image)[1]
    image = tf.image.resize_images(image, size=[img_size,img_size])
    return image_name, image, h, w
test_dataset = test_dataset.map(load_img_data)
test_iterator = test_dataset.make_one_shot_iterator()
next_test_element = test_iterator.get_next()
tdata_sess = tf.Session(config = config)

##### PREDICT & OUTPUT #####
output_file = open('./data/test_predion.txt', 'w')
solver.prepare_inference(model_version = '4000', config = config)

while True:
    try:
        img_name, test_img, img_h, img_w = tdata_sess.run(next_test_element)
        test_img = test_img/255 * 2 - 1
        test_img = np.expand_dims(test_img, axis=0)
        y_pred = solver.make_one_prediction(test_img)
        ''' 
        pred_dict = multi_process_predicts(test_img, y_pred, 0.1)
        pred_dict = non_max_suppress(pred_dict)
        for class_num, bbox in pred_dict.items():
            for bb in bbox:
                xmin, ymin, xmax, ymax, conf = bb
                xmin, ymin, xmax, ymax = xmin*(img_w/img_size), ymin*(img_h/img_size), xmax*(img_w/img_size), ymax*(img_h/img_size)
                output_file.write(img_name.decode('ascii')+" %d %d %d %d %d %f\n" %(xmin, ymin, xmax, ymax, class_num, conf))
        '''
        xmin, ymin, xmax, ymax, class_num, conf = process_predicts(y_pred)
        xmin, ymin, xmax, ymax = xmin*(img_w/img_size), ymin*(img_h/img_size), xmax*(img_w/img_size), ymax*(img_h/img_size)
        #img filename, (xmin, ymin, xmax, ymax, class, confidence)*number_of_predictions
        output_file.write(img_name.decode('ascii')+" %d %d %d %d %d %f\n" %(xmin, ymin, xmax, ymax, class_num, conf))
    except tf.errors.OutOfRangeError:
        print("done predicting all test data")
        break
output_file.close()

##### EVALUATE #####
sys.path.insert(0, './data/evaluate')
import evaluate
#evaluate.evaluate("input prediction file name", "desire output csv file name")
evaluate.evaluate('./data/test_predion.txt', './output_file.csv')

demo.py

In [ ]:
import os, cv2
import numpy as np
import matplotlib.pyplot as plt

from model_def import YoloRunner
from yolov2 import YoloTinyNet
import tensorflow as tf

def process_predicts(predicts):
    """
    Process YOLO outputs into bou
    """
    p_classes = predicts[0, :, :, 0:20]
    C = predicts[0, :, :, 20:22]
    coordinate = predicts[0, :, :, 22:]
    p_classes = np.reshape(p_classes, (net_params['cell_size'], net_params['cell_size'], 1, 20))
    C = np.reshape(C, (net_params['cell_size'], net_params['cell_size'], net_params['boxes_per_cell'], 1))
    P = C * p_classes
    #P's shape [7, 7, 2, 20]
    #print P[5,1, 0, :]
    #choose the most confidence one
    max_conf = np.max(P)
    index = np.argmax(P)
    index = np.unravel_index(index, P.shape)
    class_num = index[3]
    coordinate = np.reshape(coordinate, 
                            (net_params['cell_size'], 
                             net_params['cell_size'], 
                             net_params['boxes_per_cell'], 
                             4))
    max_coordinate = coordinate[index[0], index[1], index[2], :]
    xcenter = max_coordinate[0]
    ycenter = max_coordinate[1]
    w = max_coordinate[2]
    h = max_coordinate[3]
    xcenter = (index[1] + xcenter) * (common_params['image_size']/float(net_params['cell_size']))
    ycenter = (index[0] + ycenter) * (common_params['image_size']/float(net_params['cell_size']))
    w = w * common_params['image_size']
    h = h * common_params['image_size']
    xmin = xcenter - w/2.0
    ymin = ycenter - h/2.0
    xmax = xmin + w
    ymax = ymin + h
    return xmin, ymin, xmax, ymax, class_num, max_conf

classes_name =  ["aeroplane", "bicycle", "bird", "boat", "bottle", 
                 "bus", "car", "cat", "chair", "cow", "diningtable", 
                 "dog", "horse", "motorbike", "person", "pottedplant", 
                 "sheep", "sofa", "train","tvmonitor"]


##### SET HYPERPARAMETERS #####
common_params = {
    'image_size': 448,
    'batch_size': 64,
    'num_classes': 20,
    'max_objects_per_image': 20
}
dataset_params = {
    'path': 'data/pascal_voc_training_data.txt',
    'image_dir': './data/VOCdevkit_train/VOC2007/JPEGImages/',
    'thread_num': 6
}
net_params = {
    'weight_decay': 0.0005,
    'cell_size': 7,
    'boxes_per_cell': 2,
    'object_scale': 1,
    'noobject_scale': 0.5,
    'class_scale': 1,
    'coord_scale': 5, 

}
solver_params = {
    'learning_rate': 0.000001,
    'moment': 0.9,
    'max_iterators': 20001,
    'print_frequency': 100,
    'save_frequency' : 1000,
    'train_dir': 'models/yolo-like'
}
#if you have multiple GPU on the machine, choose only one to use on this notebook
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
#let the gpu allocates memory space dynamically
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

tf.reset_default_graph()
common_params['batch_size'] = 1
net = YoloTinyNet(common_params, net_params)
solver = YoloRunner(None, net, common_params, solver_params)
solver.prepare_inference(model_version = '25000', config = config)
test_dir = os.listdir('./data/VOCdevkit_test/VOC2007/JPEGImages/')
#cv2.namedWindow('Test', cv2.WINDOW_NORMAL)
ret = True
se = None
for name in test_dir:
    np_img = cv2.imread('./data/VOCdevkit_test/VOC2007/JPEGImages/'+name)
    #ret, np_img = cap.read()
    resized_img = cv2.resize(np_img, (common_params['image_size'], common_params['image_size']))
    np_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
    resized_img = np_img
    np_img = np_img.astype(np.float32)
    np_img = np_img / 255.0 * 2 - 1
    np_img = np.reshape(np_img, (1, common_params['image_size'], common_params['image_size'], 3))

    y_pred = solver.make_one_prediction(np_img)
    xmin, ymin, xmax, ymax, class_num, conf = process_predicts(y_pred)
    '''
    pred_dict = multi_process_predicts(resized_img, y_pred, 0.03)
    pred_dict = non_max_suppress(pred_dict)
    for name, bbox in pred_dict.items():
        for bb in bbox:    
            class_name = classes_name[name]
            xmin, ymin, xmax, ymax, conf = bb
            conf = float('%.2f' %conf)
            cv2.rectangle(resized_img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 255), 3)
            cv2.putText(resized_img, class_name+str(conf), (int(xmin), int(ymin)), 2, 1.5, (0, 255, 255), 2)
    '''
    conf = float('%.2f' %conf)  
    class_name = classes_name[class_num]
    cv2.rectangle(resized_img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 255), 3)
    cv2.putText(resized_img, class_name+str(conf), (int(xmin), int(ymin)), 2, 1.5, (0, 255, 255), 2)
    #cv2.imshow('Test', resized_img)
    if se is None:
        se = plt.imshow(resized_img)
    else:
        se.set_data(resized_img)
    plt.pause(1.0)
    plt.draw()
 
#plt.imshow(resized_img)
#plt.show()

## 3. Did you pre-train your model? If yes, what dataset did you use(briefly introduce this dataset)?


我們沒有使用pre-train model。使用的dataset是助教提供的pascal voc 2007 dataset，裡面總共有20種class，training和validation data約有10000筆，總共有24640個已標記的object。

## 4. How did you build your model(What model, any techniques you used, what obstacles you met and how did you deal with them)? 

	我們使用助教上課講解的YOLO來進行實作，一開始先試助教原本的code，發現只有fully connected的架構結果會很差，所以我們照YOLO v1的架構做完整的convolution，後來發現test出來的結果，框的區域跟判斷的種類好像都變動不大，而且loss的部分一直沒辦法收斂，而且class loss的部分特別大，所以感覺的可能是架構太深讓network沒辦法抓到它要學的東西，或是訓練參數的問題，也有可能是train的iterator不夠多，所以:
    
1.) 我們在model加入幾層residual connection，希望不會因為架構太深讓network忘了前面學到的東西。

2.) 調整訓練參數的部分，嘗試過幾種組合之後將learning rate調整成0.0001，另外，為了想降低class loss，所以我們將class_scale調整為2。

3.) 不確定loss是因為train不夠久還沒收斂還是已經不會收斂了，所以我們將iterator的部分調整為20001，結果的確有再讓loss降低一點點。

4.) 我們嘗試過將判斷class的部分改成multi-class，但發現結果反而更多錯誤，所以還是改回助教原本使用的方法。

	我們使用前面提到的方法讓training時的loss降低了許多，但testing的時候並沒有想像中的好，class loss的部分雖然有降低一些，不過沒有到理想中的低，另外，也發現它在person的class學得比較好，不確定是不是dataset中可能比較多person的圖片，應該要從前處理的部分來著手，有嘗試過圖片的flip，不過有些執行上的錯誤沒時間處理完。

## 5. Anything you think it's worth mentioning(your findings, other works). 

1. Dataset不足

Pascal voc 2007的資料有20類，總共約10000張圖片和24640筆已標記的object，這對於要訓練出一個好的model是遠遠不夠的，我們需要加上更多可以拿來做object detection的dataset，例如COCO、ImageNet等等，可惜因為我們太晚開始，導致沒有足夠的時間來做這些龐大資料量的處理成和pascal voc相同的格式，若是可以使用這些資料，我們的model做出來應該會有更好的效果。

2. Training時間不夠久

前幾名的心得分享有提到他們的training時間都是以天來計算，但我們並沒有讓model訓練那麼長的時間，可能會導致他還沒收斂好就結束。